In [111]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import joblib
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or '3' to suppress all messages
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor, XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import roc_curve, auc
label_encoder = LabelEncoder()
pd.options.mode.chained_assignment = None


# load data

In [12]:
import pickle

def save_dict_to_pickle(dictionary, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(dictionary, file)

def load_dict_from_pickle(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

In [13]:
raw_data_path = './data/data_by_table/pre_24h_data_1217.csv'
model_A_dict_file_path = './model/model_A_dict.pkl'
model_B_dict_file_path = './model/model_B_dict.pkl'
model_c_path ='./model/1219test.h5'

In [112]:
from tensorflow.keras.models import load_model
model_A_dict = load_dict_from_pickle(model_A_dict_file_path)
model_B_dict = load_dict_from_pickle(model_B_dict_file_path)
model_C = load_model('./model/1219test.h5')
scaler_C = joblib.load('./model/C_scaler.joblib')

[20:51:58] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_21wtzqx5vy/croot/xgboost-split_1675457780668/work/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[20:51:58] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_21wtzqx5vy/croot/xgboost-split_1675457780668/work/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about d

/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MLPRegressor from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [113]:
raw_data_df = pd.read_csv(raw_data_path)

print(raw_data_df.columns)

Index(['Unnamed: 0', 'charttime', 'before_weaning_hr', 'stay_id', 'subject_id',
       'O2_flow', 'heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate', 'spo2',
       'peep', 'fio2', 'tidal_volume_observed', 'respiratory_rate_set',
       'plateau_pressure', 'ventilator_mode', 'GCS', 'hadm_id', 'age_now',
       'gender', 'insurance', 'race', 'admission_type', 'first_careunit',
       'weight_kg', 'height_cm', 'tobacco', 'label', 'Rev_h', 'dod_h', 'RSBI',
       'minute_ventilation', 'ventilator_mode_group'],
      dtype='object')


# define op function

- model A 

In [85]:
def model_A_inference(test_change_label_value_aug_df, model_A_dict, feature, feature_columns, prefix):
    # Initialize an empty DataFrame to store the results
    regression_results = pd.DataFrame(index=test_change_label_value_aug_df.index)

    classifier = model_A_dict[feature]['classifier']
    regressor = model_A_dict[feature]['regressor']
    scaler_classifier = model_A_dict[feature]['scaler_classifier']
    scaler_regressor = model_A_dict[feature]['scaler_regressor']

    test_scaled_df = scaler_classifier.transform(test_change_label_value_aug_df[feature_columns])

    # Make classification predictions
    classification_predict = classifier.predict(test_scaled_df)
    if(classification_predict[0] == 0):
        test_change_label_value_aug_df.reset_index
        return test_change_label_value_aug_df[feature].values[0]
    #print_classification_metrics(test_change_label_value_aug_df[f'{feature}_change_label'], classification_predict, prefix)

    # Filter rows based on classification prediction
    classification_0_rows = test_change_label_value_aug_df[classification_predict == 0]

    # Set regression_predict for rows with classification 0
    regression_results.loc[classification_0_rows.index, f'{feature}_regression_predict'] = \
        test_change_label_value_aug_df.loc[classification_0_rows.index, f'{feature}_prev_1h']

    # Set regression_predict for rows with classification 1 using regressor predictions
    classification_1_rows = test_change_label_value_aug_df[classification_predict == 1]
    test_scaled_df = scaler_regressor.transform(test_change_label_value_aug_df.loc[classification_1_rows.index, feature_columns])
    regression_predictions = regressor.predict(test_scaled_df)
    regression_results.loc[classification_1_rows.index, f'{feature}_regression_predict'] = regression_predictions
    return regression_predictions
    
    #print_regression_metrics(test_change_label_value_aug_df[feature], regression_results, prefix)

In [72]:
def get_A_one_row_data(df, stay_id, before_weaning_hr):
    # Filter the DataFrame based on stay_id and before_weaning_hr
    filtered_df = df[(df['stay_id'] == stay_id) & (df['before_weaning_hr'] == before_weaning_hr)].copy()

    # Select the desired feature columns
    feature_columns = [
        'before_weaning_hr', 'age_now', 'weight_kg',
        'height_cm', 'tobacco', 'heart_rate', 'sbp', 'dbp', 'mbp', 'spo2', 'resp_rate',
        'tidal_volume_observed', 'RSBI', 'minute_ventilation', 'peep', 'fio2',
        'respiratory_rate_set', 'plateau_pressure'
    ]

    # Add columns for previous 1-hour values
    prev_1h_columns = [
        'heart_rate', 'sbp', 'dbp', 'mbp', 'spo2', 'resp_rate',
        'tidal_volume_observed', 'RSBI', 'minute_ventilation', 'peep', 'fio2',
        'respiratory_rate_set', 'plateau_pressure'
    ]

    for col in prev_1h_columns:
        col_name_prev_1h = f'{col}_prev_1h'
        # Get the previous 1-hour value from the same stay_id but at before_weaning_hr + 1
        filtered_df[col_name_prev_1h] = df[(df['stay_id'] == stay_id) & (df['before_weaning_hr'] == before_weaning_hr + 1)][col].values

    for col in prev_1h_columns:
        col_name_prev_1h = f'{col}_prev_2h'
        # Get the previous 1-hour value from the same stay_id but at before_weaning_hr + 1
        filtered_df[col_name_prev_1h] = df[(df['stay_id'] == stay_id) & (df['before_weaning_hr'] == before_weaning_hr + 2)][col].values

    # Return the final DataFrame with selected columns
    return filtered_df

def get_B_one_row_data(df, stay_id, before_weaning_hr, features):
    # Filter the DataFrame based on stay_id and before_weaning_hr
    filtered_df = df[(df['stay_id'] == stay_id) & (df['before_weaning_hr'] == before_weaning_hr)].copy()

    # Select the desired feature columns
    feature_columns = [
        'before_weaning_hr', 'age_now', 'weight_kg',
        'height_cm', 'tobacco', 'heart_rate', 'sbp', 'dbp', 'mbp', 'spo2', 'resp_rate',
        'tidal_volume_observed', 'RSBI', 'minute_ventilation', 'peep', 'fio2',
        'respiratory_rate_set', 'plateau_pressure'
    ]

    # Add columns for previous 1-hour values
    prev_1h_columns = [
        'heart_rate', 'sbp', 'dbp', 'mbp', 'spo2', 'resp_rate',
        'tidal_volume_observed', 'RSBI', 'minute_ventilation', 'peep', 'fio2',
        'respiratory_rate_set', 'plateau_pressure'
    ]

    for col in prev_1h_columns:
        col_name_prev_1h = f'{col}_prev_1h'
        # Get the previous 1-hour value from the same stay_id but at before_weaning_hr + 1
        filtered_df[col_name_prev_1h] = df[(df['stay_id'] == stay_id) & (df['before_weaning_hr'] == before_weaning_hr + 1)][col].values

    for col in prev_1h_columns:
        col_name_prev_1h = f'{col}_prev_2h'
        # Get the previous 1-hour value from the same stay_id but at before_weaning_hr + 1
        filtered_df[col_name_prev_1h] = df[(df['stay_id'] == stay_id) & (df['before_weaning_hr'] == before_weaning_hr + 2)][col].values

    # Return the final DataFrame with selected columns
    return filtered_df

In [86]:
feature_columns = [
    'before_weaning_hr', 'age_now', 'weight_kg',
    'height_cm', 'tobacco', 'heart_rate', 'sbp', 'dbp', 'mbp', 'spo2', 'resp_rate', 'tidal_volume_observed', 'RSBI',
    'minute_ventilation', 'heart_rate_prev_1h', 'sbp_prev_1h', 'dbp_prev_1h', 'mbp_prev_1h', 'spo2_prev_1h',
    'resp_rate_prev_1h', 'tidal_volume_observed_prev_1h', 'RSBI_prev_1h', 'minute_ventilation_prev_1h', 
    'peep_prev_1h', 'fio2_prev_1h', 'respiratory_rate_set_prev_1h', 'plateau_pressure_prev_1h',
]

In [ ]:
['heart_rate_prev_1h', 'sbp_prev_1h',
 'dbp_prev_1h', 'mbp_prev_1h', 'spo2_prev_1h',
 'resp_rate_prev_1h', 'tidal_volume_observed_prev_1h',
 'RSBI_prev_1h', 'minute_ventilation_prev_1h', 'peep_prev_1h', 
 'fio2_prev_1h', 'respiratory_rate_set_prev_1h', 'plateau_pressure_prev_1h']

- A test Tester

In [88]:
result_df = get_A_one_row_data(raw_data_df, 34670891, 13)
haha = model_A_inference(result_df, model_A_dict, 'plateau_pressure', feature_columns, 'whole set')
print(haha)

26.0


- model B

In [107]:
def model_B_inference(test_change_label_value_aug_df, model_B_dict, feature, feature_columns, prefix):
    regressor = model_B_dict[feature]
    regression_predictions = regressor.predict(test_change_label_value_aug_df[feature_columns])
    return regression_predictions[0]

In [100]:
model_B_feature_columns = ['before_weaning_hr', 'age_now', 'weight_kg', 'height_cm', 'tobacco', # 'gender', 
          'peep', 'fio2', 'respiratory_rate_set', 'plateau_pressure',
          'peep_prev_1h', 'fio2_prev_1h', 'respiratory_rate_set_prev_1h', 'plateau_pressure_prev_1h',
          'heart_rate_prev_1h', 'sbp_prev_1h', 'dbp_prev_1h',
          'mbp_prev_1h', 'spo2_prev_1h', 'resp_rate_prev_1h', 'tidal_volume_observed_prev_1h', 'RSBI_prev_1h', 'minute_ventilation_prev_1h',
          'heart_rate_prev_2h', 'sbp_prev_2h', 'dbp_prev_2h',
          'mbp_prev_2h', 'spo2_prev_2h', 'resp_rate_prev_2h', 'tidal_volume_observed_prev_2h', 'RSBI_prev_2h', 'minute_ventilation_prev_2h']

In [92]:
vitalsign = ['heart_rate', 'sbp', 'dbp', 'mbp', 'spo2', 'resp_rate', 
             'tidal_volume_observed', 'RSBI', 'minute_ventilation']
ventilator_settings = ['peep', 'fio2', 'respiratory_rate_set', 'plateau_pressure']
baseline = ['age_now','gender', 'insurance', 'race', 'admission_type', 'first_careunit'
,'weight_kg', 'height_cm', 'tobacco' ]
result_df = get_B_one_row_data(raw_data_df, 34670891, 13, model_B_feature_columns)

In [108]:
haha2 = model_B_inference(result_df, model_B_dict, 'heart_rate', model_B_feature_columns, 'whole set')
print(haha2)

80.7755


# ABC model

In [ ]:
def save_a_patient(df ,stay_id, start_save_hr):
    patient_df = patient_df[patient_df['stay_id']==stay_id]
    result_df = get_A_one_row_data(df, stay_id, 13)